In [147]:
import pandas as pd
import numpy as np
from sklearn import metrics
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold, train_test_split

In [2]:
df = pd.read_excel('dataBaseTratada.xlsx')

df.head()

,product_category_name,product_category_name_english,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_item_id,product_id,...,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,Latitude_Comprador,Longitude_Comprador,Latitude_Vendedor,Longitude_Vendedor,Distancia
0,utilidades_domesticas,housewares,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,...,8.0,13.0,9350,maua,SP,-23.5329,-46.6395,-23.6677,-46.4613,23.545096
1,utilidades_domesticas,housewares,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,...,8.0,13.0,9350,maua,SP,-23.5329,-46.6395,-23.6677,-46.4613,23.545096
2,utilidades_domesticas,housewares,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sao paulo,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,...,8.0,13.0,9350,maua,SP,-23.5329,-46.6395,-23.6677,-46.4613,23.545096
3,papelaria,stationery,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,sao paulo,ad21c59c0840e6cb83a9ceb5573f8159,1,65266b2da20d04dbe00c5c2d3bb7859e,...,15.0,15.0,8752,mogi das cruzes,SP,-23.6737,-46.5432,-23.5208,-46.1854,40.228060
4,automotivo,auto,503740e9ca751ccdda7ba28e9ab8f608,80bb27c7c16e8f973207a5086ab329e2,86320,congonhinhas,parana,a4591c265e18cb1dcee52889e2d8acc3,1,060cb19345d90064d1015407193c233d,...,10.0,65.0,7112,guarulhos,SP,-23.4969,-50.6455,-23.4538,-46.5333,419.422496


In [3]:
df.shape

(73400, 46)

In [4]:
df.describe

<bound method NDFrame.describe of         product_category_name product_category_name_english  \
0       utilidades_domesticas                    housewares   
1       utilidades_domesticas                    housewares   
2       utilidades_domesticas                    housewares   
3                   papelaria                    stationery   
4                  automotivo                          auto   
...                       ...                           ...   
73395                   bebes                          baby   
73396      eletrodomesticos_2             home_appliances_2   
73397  informatica_acessorios         computers_accessories   
73398  informatica_acessorios         computers_accessories   
73399            beleza_saude                 health_beauty   

                            customer_id                customer_unique_id  \
0      9ef432eb6251297304e76186b10a928d  7c396fd4830fd04220f754e42b4e5bff   
1      9ef432eb6251297304e76186b10a928d  7c396fd4830fd0

In [17]:
X_df = df[['price', 'payment_value', 'freight_value', 'Distancia', 'product_weight_g', 'product_width_cm', 'product_height_cm', 'product_length_cm']]
Y_df = df['delivery_time']

In [43]:
X_df_train, X_df_test, Y_df_train, Y_df_test = train_test_split(X_df, Y_df, random_state=42)

In [19]:
clf = xgb.XGBRegressor()
clf.objective

'reg:squarederror'

In [20]:
#regressor padrao
regressor_xgb = xgb.XGBRegressor()

In [21]:
regressor_xgb.objective

'reg:squarederror'

In [22]:
cross_val_score(regressor_xgb, X_df_train, Y_df_train, scoring='neg_root_mean_squared_error').mean()

-7.594767826632783

In [23]:
#regressor com limite de profundidade (ruim)
regressor_xgb_tunado = xgb.XGBRegressor(max_depth=2)

cross_val_score(regressor_xgb_tunado, X_df_train, Y_df_train, scoring='neg_root_mean_squared_error').mean()

-7.906211563976439

In [24]:
#apenas regressão linear (ruim)
regressor_xgb_gblinear =  xgb.XGBRegressor(booster='gblinear')

cross_val_score(regressor_xgb_gblinear, X_df_train, Y_df_train, scoring='neg_root_mean_squared_error').mean()

-8.052231949867068

In [101]:
X_df_categorico = df[['customer_state', 'product_category_name', 'seller_state', 'price', 'payment_value', 'freight_value', 'Distancia', 'product_weight_g', 'product_width_cm', 'product_height_cm']]

X_df_categorico_train, X_df_categorico_teste, Y_df_train, Y_df_test = train_test_split(X_df_categorico, Y_df, random_state=42)

In [146]:
#variavel categorica + gbtree

for col in X_df_categorico_train.select_dtypes(include=['object']).columns:
    X_df_categorico_train[col] = X_df_categorico_train[col].astype('category')

regressor_1 =  xgb.XGBRegressor(n_estimators=100, num_parallel_tree=3, enable_categorical=True, booster='gbtree', max_bin=512)

cross_val_score(regressor_1, X_df_categorico_train, Y_df_train, scoring='neg_root_mean_squared_error').mean()

-7.479221300030193

In [151]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from numpy import mean
from numpy import std

forest_clf = RandomForestClassifier()

search_space = {"bootstrap": Categorical([True]), 
        "max_depth": Integer(6, 20),  
        "min_samples_leaf": Integer(2, 10),
        "min_samples_split": Integer(2, 10),
        "n_estimators": Integer(100, 500)
        }

forest_bayes_search = BayesSearchCV(forest_clf, search_space, n_iter=32,
                                    scoring="accuracy", n_jobs=-1, cv=5)

forest_bayes_search.fit(X_df_categorico_train, Y_df_train)

ValueError: could not convert string to float: 'minas gerais'

In [103]:
#variaveis categoricas + dart (igual ao gbtree só que mais demorado)
for col in X_df_categorico_train.select_dtypes(include=['object']).columns:
    X_df_categorico_train[col] = X_df_categorico_train[col].astype('category')

regressor_2 =  xgb.XGBRegressor(enable_categorical=True, booster='dart', max_bin=512)

cross_val_score(regressor_2, X_df_categorico_train, Y_df_train, scoring='neg_root_mean_squared_error').mean()

-7.480179533982098

In [152]:
regressor_campeao = regressor_1

regressor_campeao = xgb.XGBRegressor(eval_metric='rmsle')

regressor_campeao.fit(X_df_train, Y_df_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmsle', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [153]:
predicoes = regressor_campeao.predict(X_df_test)

predicoes[:10]

array([13.70811 , 11.303275, 17.320446, 10.498982, 14.258818, 15.214599,
       10.421895, 10.250247, 11.952857, 14.704144], dtype=float32)

In [154]:
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import r2_score

mse = mean_squared_error(Y_df_test, predicoes)

display(mse)

rmse = math.sqrt(mse)

display(rmse)

r2 = r2_score(Y_df_test, predicoes)

display(r2)

54.11449422260647

7.356255448433426

0.1854188711608864